# Proposal 3

$$
\mathcal{L} = \mathcal{L}_c + \lambda_e \cdot || \bm{p^*_A} \odot \bm{(\theta - \theta^*_A) }||_1 + \lambda_f \cdot || \bm{(1 - p)} \odot \bm{\theta} ||_1
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    forgetting_metrics,
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda, device

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import MWUNV1
from base_code.seed import set_seed

# Constants

In [5]:
TORCH_DEVICE = device("cuda" if cuda.is_available() else "cpu")
N_EXPERIENCES = 50
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_3.pkl"
set_seed(EXPERIMENT_SEED)

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'lambda_e': 10.0,
    'lambda_f': 10.0,
    'eps': 0.0001
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),

    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = MWUNV1(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/50 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 78.56it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:05<00:00, 78.72it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:05<00:00, 78.41it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:05<00:00, 78.36it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:05<00:00, 78.97it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 102.50it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7692
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 103.22it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1882
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 100.34it/s]
> Eval

  2%|▏         | 1/50 [01:06<54:37, 66.88s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1067
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1532
-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 90.69it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:05<00:00, 91.95it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:05<00:00, 91.18it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:05<00:00, 92.73it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:05<00:00, 91.22it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 121.53it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0467
	ExperienceForgetting/eval_phase/test_stream

  4%|▍         | 2/50 [02:05<49:50, 62.30s/it]

...


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1156
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.3538
	StreamForgetting/eval_phase/test_stream = 0.3538
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.4840
-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 93.48it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:05<00:00, 90.53it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:05<00:00, 93.66it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 94.68it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 94.04it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 116.16it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.5538
	ExperienceForgetting/eval_phase/test_strea

 98%|█████████▊| 49/50 [59:44<00:58, 58.66s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.1282
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.3628
	StreamForgetting/eval_phase/test_stream = 0.3628
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.4835
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 94.02it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 94.18it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:05<00:00, 93.41it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 94.12it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 94.06it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 117.73it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.5769
	ExperienceForgetting/eval_phase/test_strea

100%|██████████| 50/50 [1:00:43<00:00, 72.88s/it]


> Eval on experience 49 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp049 = 0.8586
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.3802
	StreamForgetting/eval_phase/test_stream = 0.3802
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.4738


# Get metrics

In [12]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [13]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))